In [15]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from datetime import datetime

In [17]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import matplotlib as plt
import numpy as np
import pandas as pd

image_paths = ['/kaggle/input/dip-lab-hackathon-2024-image-classification/data/train', '/kaggle/input/dip-lab-hackathon-2024-image-classification/data/test']

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),    
    transforms.RandomHorizontalFlip(),    
    transforms.RandomRotation(15),        
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
    transforms.ToTensor(),                
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),   
])

val_transform = transforms.Compose([
    transforms.Resize(256),              
    transforms.CenterCrop(224),           
    transforms.ToTensor(),               
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

Ds_target = datasets.ImageFolder(root='/kaggle/input/dip-lab-hackathon-2024-image-classification/data/train', transform=train_transform)

train_size = int(0.9 * len(Ds_target))
val_size = len(Ds_target) - train_size
training_set, validation_set = torch.utils.data.random_split(Ds_target, [train_size, val_size])

validation_set.dataset.transform = val_transform

training_loader = DataLoader(training_set, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size=64, shuffle=False)

print(f"Training set size: {len(training_set)}")
print(f"Validation set size: {len(validation_set)}")

Training set size: 4502
Validation set size: 501


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import time
from torchvision import models

# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained DenseNet201 model
densenet169 = models.densenet169(pretrained=True)

# Freeze all layers except the classifier
for param in densenet169.parameters():
    param.requires_grad = True

# Update the classifier to match the number of output classes (e.g., 10)
num_classes = 10
densenet169.classifier = nn.Linear(densenet169.classifier.in_features, num_classes)

# Move the entire model to the correct device (GPU/CPU)
densenet169 = densenet169.to(device)

# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(densenet169.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

epoch_number = 0
EPOCHS = 20
path_save_cp = './cp/'
best_vloss = 1_000_000.
training_logs = {"train_loss": [], "train_acc": [], "validate_loss": [], "validate_acc": []}

# Start the training loop
t_0_accelerated = time.time()
for epoch in range(EPOCHS):
    train_loss, train_correct = 0, 0

    # Set model to training mode
    densenet169.train(True)

    # Iterate through training data
    for i, data in enumerate(training_loader):
        inputs, labels = data[0].to(device), data[1].to(device)  # Move inputs and labels to the device

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Forward pass: Make predictions
        outputs = densenet169(inputs)

        # Compute loss and backpropagate
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Update weights
        optimizer.step()

        train_loss += loss.item()
        train_correct += (outputs.argmax(1) == labels).float().sum().item()

    # Step the scheduler
    scheduler.step()

    # Log training loss and accuracy
    training_logs["train_loss"].append(train_loss / len(training_loader))
    training_logs["train_acc"].append(train_correct / len(training_loader.dataset))

    # Validation
    running_vloss = 0.0
    densenet169.eval()  # Set model to evaluation mode
    valid_loss, valid_correct = 0, 0
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata[0].to(device), vdata[1].to(device)  # Move validation data to device
            voutputs = densenet169(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            valid_loss += vloss.item()
            valid_correct += (voutputs.argmax(1) == vlabels).float().sum().item()

    # Log validation loss and accuracy
    training_logs["validate_loss"].append(valid_loss / len(validation_loader))
    training_logs["validate_acc"].append(valid_correct / len(validation_loader.dataset))

    # Print progress
    if epoch % 1 == 0:
        print(f"Epoch {epoch+1}".ljust(10),
              f"Train Loss: {training_logs['train_loss'][-1]:.5f}",
              f"Train Acc: {training_logs['train_acc'][-1]:.5f}",
              f"Validation Loss: {training_logs['validate_loss'][-1]:.5f}",
              f"Validation Acc: {training_logs['validate_acc'][-1]:.5f}")
        print("-" * 80)

    # Save best model
    if valid_loss < best_vloss:
        best_vloss = valid_loss
        if not os.path.exists(path_save_cp):
            os.mkdir(path_save_cp)
        torch.save(densenet169.state_dict(), path_save_cp + 'best_pretrainedmodel.pth')

    epoch_number += 1

t_end_accelerated = time.time() - t_0_accelerated
print(f"Time consumption for accelerated CUDA training (device:{device}): {t_end_accelerated} sec")


Epoch 1    Train Loss: 2.10680 Train Acc: 0.25322 Validation Loss: 1.66864 Validation Acc: 0.50699
--------------------------------------------------------------------------------
Epoch 2    Train Loss: 1.22800 Train Acc: 0.65260 Validation Loss: 0.86183 Validation Acc: 0.75449
--------------------------------------------------------------------------------
Epoch 3    Train Loss: 0.66435 Train Acc: 0.80675 Validation Loss: 0.51884 Validation Acc: 0.83832
--------------------------------------------------------------------------------
Epoch 4    Train Loss: 0.40147 Train Acc: 0.88272 Validation Loss: 0.43084 Validation Acc: 0.86427
--------------------------------------------------------------------------------
Epoch 5    Train Loss: 0.24244 Train Acc: 0.93025 Validation Loss: 0.38352 Validation Acc: 0.89022
--------------------------------------------------------------------------------
Epoch 6    Train Loss: 0.14250 Train Acc: 0.96690 Validation Loss: 0.36092 Validation Acc: 0.89820
-

KeyboardInterrupt: 

In [20]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn = torch.nn.CrossEntropyLoss()

PATH = '/kaggle/working/cp/best_pretrainedmodel.pth'


model = models.densenet169(pretrained=False)
model.classifier = torch.nn.Linear(model.classifier.in_features, 10)

model.load_state_dict(torch.load(PATH, map_location=device), strict=False)

model.to(device)

model.eval()

acc_test = 0
test_loss = 0


with torch.no_grad():
    for tinputs, tlabels in validation_loader:
        tinputs, tlabels = tinputs.to(device), tlabels.to(device)


        toutputs = model(tinputs)


        loss = loss_fn(toutputs, tlabels)
        test_loss += loss.item()

 
        _, preds_t = torch.max(toutputs, 1)
        acc_test += (preds_t == tlabels).float().sum().item()

accuracy_t = round(acc_test / len(validation_loader.dataset) * 100, 2)
avg_tloss = test_loss / len(validation_loader)

print(f'[Test loss: {avg_tloss}] [Accuracy test: {accuracy_t}%]')


/tmp/ipykernel_36/1110901835.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(PATH, map_location=device), strict=False)


[Test loss: 0.35990913584828377] [Accuracy test: 90.22%]


In [21]:
import pandas as pd

model.eval()
df = pd.read_csv("/kaggle/input/dip-lab-hackathon-2024-image-classification/sample-submission.csv")
image_dir = '/kaggle/input/dip-lab-hackathon-2024-image-classification/data/test/data'
predictions = []

for idx, row in df.iterrows():
    image_path = os.path.join(image_dir, row['ID'])
    image = Image.open(image_path)
    
    image = val_transform(image).unsqueeze(0)
    
    if torch.cuda.is_available():
        image = image.cuda()
        loaded_model = model.cuda()
    
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    predictions.append(predicted.item())

df['predicted_class'] = predictions

df.to_csv('/kaggle/working/submission.csv', index=False)

print("CSV file updated with predictions.")

CSV file updated with predictions.
